# Some example demographies

We illustrate some advanced options of `momi.Demography()`.

In [1]:
from momi import Demography, to_ms_cmd

### Specifying demographies in different time scales

Users can choose to input demographies in ms time scale, the more 'standard' coalescent time scale, or in per-generation units.

In [2]:
# a simple Demography, in ms units
events = [('-eg', 0.0, 'a', 1.0), ('-ej', .5, 'a', 'b'), ('-en', 1., 'b', .5)]
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3))

print "Demography in ms syntax:"
print to_ms_cmd(demo)

Demography in ms syntax:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [3]:
# the same Demography, but specified in the 'standard' (not 'ms') time scale
# this means that there are 2*N_ref generations per unit time (instead of 4*N_ref as in ms)

# notice times are 2x longer in the 'standard' units.
# as consequence, growth rate per unit time is 2x slower
events = [('-eg', 0.0, 'a', 0.5), ('-ej', 1., 'a', 'b'), ('-en', 2., 'b', .5)]

# use time_scale='standard' to specify standard coalescent units
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3), time_scale='standard')

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


In [4]:
# the same Demography, but specified in per-generation units
# we'll assume a diploid population size of 10,000

# join event 5e3 generations ago. population size drops to 5000 at 1e4 generations ago.
# growth rate per generation is 1e-4 in population 'a'
events = [('-eg',0.0,'a',1e-4),('-ej', 5e3, 'a', 'b'), ('-en', 1e4, 'b', 5000)]

# use default_N to set the diploid population size
demo = Demography(events, sampled_pops=('a','b'), sampled_n=(5,3), default_N=10000)

try:
    print to_ms_cmd(demo)
except:
    print "Must convert demography to ms units before calling to_ms_cmd()\n"

demo = demo.rescaled()
print "Demography in ms syntax & units:"
print to_ms_cmd(demo)

Must convert demography to ms units before calling to_ms_cmd()

Demography in ms syntax & units:
-I 2 5 3 -eg 0.0 1 1.0 -ej 0.5 1 2 -en 1.0 2 0.5


### Ghost populations

A 'ghost' population is one that is not present in the sample.

`momi` syntax is more flexible than `ms` about ghost populations. In particular, ghost populations can be immediately affected by demographic events.

By contrast, in `ms`, ghost populations cannot be modified by events until they have been "created" with the **-es** flag.

In [5]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0), # can set growth rate of 'z' at t=0, even tho its a ghost population
          ('-ep', .5, 'a', 'z', .25), # pulse migration from z to a
          ('-ej', 1.0, 'z', 'a')]

demo = Demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the growth event (-eg) doesn't happen until the admixture event (-es)

Demography in ms syntax.
-I 1 10 -es 0.5 1 0.75 -en 0.5 2 0.606530659713 -eg 0.5 2 1.0 -ej 1.0 2 1


In `momi`, we can also have sampled populations merge **-ej** with ghost populations, even if there is no admixture event.

In [6]:
# a demography with one sampled population 'a', and a ghost population 'z'

events = [('-eg', 0.0, 'z', 1.0),
          ('-ej', .5, 'a', 'z'), # 'z' is empty, so this is equivalent to just changing size/growth of 'a'
          ('-en', 1.0, 'z', 1.0)]

demo = Demography(events, ['a'], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)
# Note in ms syntax, the -ej event doesn't appear; instead just the size and growth rate of 'a' change

Demography in ms syntax.
-I 1 10 -en 0.5 1 0.606530659713 -eg 0.5 1 1.0 -en 1.0 1 1.0


### Ancient DNA

Use the `sampled_t` parameter to indicate that a population was sampled in the past.

In [7]:
events = [('-ej', 1.0, 'a', 'b')]

# the samples from 'b' are ancient DNA, from time t=0.5
demo = Demography(events, ['a','b'], (6, 10), sampled_t=[0.0, .5])

try:
    print to_ms_cmd(demo)
except:
    print "Cannot convert demography with archaic populations to ms syntax"

Cannot convert demography with archaic populations to ms syntax


### Demography with a lot of parameters

An example on how to programatically construct a demography with a lot of parameters (too many to conveniently write down by hand).

In [8]:
# piecewise constant demography, with 50 pieces

import random

n_pieces = 50
t = 0.0
events = []
for i in range(n_pieces):
    t = t + random.expovariate(100.)
    N = random.expovariate(1.0)
    events.append(('-en', t, 0, N))

demo = Demography(events, [0], [10])
print "Demography in ms syntax."
print to_ms_cmd(demo)

Demography in ms syntax.
-I 1 10 -en 0.00502733592733 1 0.72163578879 -en 0.0119173051205 1 0.0571889835641 -en 0.0136403786741 1 0.854163265534 -en 0.0245373040639 1 0.177603175774 -en 0.0619075080941 1 1.28757619132 -en 0.0740844048851 1 0.0674557960961 -en 0.081133032925 1 0.709446670247 -en 0.0870417103919 1 0.0266135480216 -en 0.0928914629674 1 1.70653610769 -en 0.0964226470305 1 0.390301726362 -en 0.0984005686298 1 0.615083353873 -en 0.105824677369 1 0.105027746903 -en 0.108338445097 1 1.71631352441 -en 0.122752141847 1 0.407083207599 -en 0.126141229497 1 0.406852011486 -en 0.138005033156 1 2.06819854549 -en 0.138201251129 1 1.28828339636 -en 0.146197428532 1 3.43464695824 -en 0.149630663115 1 0.0413543794081 -en 0.154779692725 1 0.412470571452 -en 0.155498107168 1 1.00298398128 -en 0.156514811215 1 0.533124531087 -en 0.170301191226 1 0.498134997861 -en 0.175260747352 1 0.370858212925 -en 0.182044165064 1 1.28570116773 -en 0.186424183162 1 2.59587315768 -en 0.199859528979 1 2.486